In [3]:
import numpy as np
import pandas as pd
import os
pd.set_option("display.max_columns", 99)
os.chdir("/home/joosungm/projects/def-lelliott/joosungm/projects/ssc23-case-comp")
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("/content/drive/MyDrive/projects/ssc23-case-comp")

## Production data

We first prepare the production data by extracting year-end(month==12) production data from all census subdivisions.

* We pick month == 12 instead of yearly sums because we believe the year-end production best reflects the consequences of extreme weather during the year than yearly sum.

In [4]:
province_prod = pd.DataFrame()
provinces = ["AB", "BC", "MB", "NB", "NL", "NS", "ON", "PE", "QC", "PE", "SK"]
for pr in provinces:
    
    prod_temp_filename = "./data/user_data/01_iv_analysis/" + pr + "/prod_temp.csv"

    prod_temp = pd.read_csv(prod_temp_filename)
    
    # - Exclude "production_in_division_" from all column names
    prod_temp.columns = [col.replace("production_in_division_", "") for col in prod_temp.columns]
    
    # - Rename lat, lon to prod_lat, prod_lon
    prod_temp.rename(columns = {"lat":"prod_lat", "long":"prod_lon"}, inplace = True)
    
    # - Extract data for month == 12.
    prod_temp = prod_temp.loc[prod_temp.month == 12, :].drop(columns = ["Date", "month", "tavg", "tmin", "tmax", "census_year_ref", "max_lat", "min_lat", "max_long", "min_long"]).reset_index(drop = True)

    # - collect prod_temp by concat prod_temp to province_prod by row
    province_prod = pd.concat([province_prod, prod_temp], axis = 0)

print(province_prod.shape)
province_prod.tail()

# drop rows that have NaN Dominant_NAICS
province_prod = province_prod.dropna(subset = ["Dominant_NAICS"])
province_prod.tail()

# save province_prod
province_prod.to_csv("./data/user_data/02_counterfactual_analysis/province_prod.csv", index = False)

(129150, 22)


In [5]:
# - Check column names 
province_prod.columns

Index(['provincename', 'X22.Utilities', 'X23.Construction',
       'X31.33.Manufacturing', 'X48.49.Transportation.and.warehousing',
       'X61.Educational.services', 'X62.Health.care.and.social.assistance',
       'X72.Accommodation.and.food.services',
       'X81.Other.services..except.public.administration.',
       'X91.Public.administration',
       'X11.Agriculture.forestry.fishing.hunting.21.Mining.quarrying.and.oil.and.gas.extraction',
       'X41.Wholesale.trade.44.45.Retail.trade',
       'X52.Finance.and.insurance.53.Real.estate.and.rental.and.leasing',
       'X54.Professional..scientific.and.technical.services.55.56',
       'X51.Information.culture.and.recreation.71', 'Population', 'GeoUID',
       'Dominant_NAICS', 'colourval', 'year', 'prod_lat', 'prod_lon'],
      dtype='object')

## Weather station data

Here, we create a dataset that includes:

* name, lat, long of the weather stations.
* year
* tmin
* tmax
* tavg
* prcp
<!-- * extreme_flag: total number of either of ext_tmax or ext_tmin during the year. -->
* tmax_flag: total number of ext_tmax during the year.
* tmin_flag: total number of ext_tmin during the year.
* total_flag: sum of tmax_flag and tmin_flag.



In [6]:
# Load data
weather = pd.read_csv("./data/climate_data/weather_Station_data.csv")
print(weather.columns)

# Extract columns
weather_columns = ["Station Name", "Longitude (x)", "Latitude (y)", "Year", "Month", 
"Mean Max Temp (°C)", 'Mean Min Temp (°C)', 'Mean Temp (°C)', 
'Extr Max Temp (°C)', 'Extr Max Temp Flag', 'Extr Min Temp (°C)', 'Extr Min Temp Flag',
'Total Precip (mm)', 'Total Precip Flag'
]

weather = weather.loc[:, weather_columns]
# display(weather.head())
weather2 = weather.rename(columns = {
    "Longitude (x)":"lon", "Latitude (y)":"lat", "Year":"year", "Month":"month",
    "Mean Max Temp (°C)":"tmax", "Mean Min Temp (°C)":"tmin", "Mean Temp (°C)":"tavg", 
    "Extr Max Temp (°C)":"tmax_ext", "Extr Min Temp (°C)":"tmin_ext",
    "Extr Max Temp Flag":"tmax_ext_flag", "Extr Min Temp Flag":"tmin_ext_flag",
    "Total Precip (mm)":"precip", "Total Precip Flag":"precip_flag"})
weather2.sample(5)

Index(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID',
       'Date/Time', 'Year', 'Month', 'Mean Max Temp (°C)',
       'Mean Max Temp Flag', 'Mean Min Temp (°C)', 'Mean Min Temp Flag',
       'Mean Temp (°C)', 'Mean Temp Flag', 'Extr Max Temp (°C)',
       'Extr Max Temp Flag', 'Extr Min Temp (°C)', 'Extr Min Temp Flag',
       'Total Rain (mm)', 'Total Rain Flag', 'Total Snow (cm)',
       'Total Snow Flag', 'Total Precip (mm)', 'Total Precip Flag',
       'Snow Grnd Last Day (cm)', 'Snow Grnd Last Day Flag',
       'Dir of Max Gust (10's deg)', 'Dir of Max Gust Flag',
       'Spd of Max Gust (km/h)', 'Spd of Max Gust Flag'],
      dtype='object')


/tmp/ipykernel_1460032/725360593.py:2: DtypeWarning: Columns (17,21,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv("./data/climate_data/weather_Station_data.csv")


,Station Name,lon,lat,year,month,tmax,tmin,tavg,tmax_ext,tmax_ext_flag,tmin_ext,tmin_ext_flag,precip,precip_flag
52857,SUN PEAKS UPPER,-119.92,50.90,2006,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6996,RIVIERE AU TONNERRE,-64.78,50.28,2011,8,18.1,11.8,14.9,23.0,E,6.0,E,149.3,E
16123,MAGOG,-72.12,45.27,2001,2,-3.8,-14.6,-9.2,8.0,NaN,-25.0,NaN,74.8,NaN
37726,SWIFT CURRENT CDA,-107.73,50.27,2004,5,14.7,2.4,8.6,24.3,NaN,-4.2,NaN,83.7,NaN
15127,RICHMOND,-72.13,45.63,1998,10,12.3,4.3,8.3,24.0,NaN,-5.5,NaN,67.2,NaN


In [7]:
print(weather2.tmax_ext_flag.unique())
print(weather2.tmin_ext_flag.unique()) 
print(weather2.precip_flag.unique())
# I: incomplete
# S: more than 1 occurrence
# E: estimated
# B: more than 1 occurence & estimated
# M: missing
# T: trace; value is zero.


[nan 'I' 'S' 'E' 'B']
[nan 'S' 'I' 'E' 'B' 'M']
[nan 'E' 'I' 'M' 'T' 'TRUE']


In [8]:
# extreme_flag == 1 if one of tmax_ext_flag, tmin_ext_flag is either one of S, E, and B
weather2["extreme_flag"] = np.where((weather2.tmax_ext_flag.isin(["S", "E", "B"])) | (weather2.tmin_ext_flag.isin(["S", "E", "B"])), 1, 0)

# tmax_flag == 1 if tmax_ext_flag is either one of S, E, and B
weather2["tmax_flag"] = np.where(weather2.tmax_ext_flag.isin(["S", "E", "B"]), 1, 0)

# tmin_flag == 1 if tmin_ext_flag is either one of S, E, and B
weather2["tmin_flag"] = np.where(weather2.tmin_ext_flag.isin(["S", "E", "B"]), 1, 0)

# total_flag = sum of tmax_flag, tmin_flag, and precip_flag
weather2["total_flag"] = weather2.tmax_flag + weather2.tmin_flag

# group by Station Name, year, lat, long, and sum the flags.
weather3 = weather2.groupby(["Station Name", "year", "lat", "lon"], as_index=False).agg({"extreme_flag":"sum", "tmax_flag":"sum", "tmin_flag":"sum", "total_flag":"sum"})
# weather3.head()

# filter out unique sets of lat, lon
weather_unique = weather3.drop_duplicates(subset = ["lat", "lon"]).reset_index(drop = True)[["Station Name", "lat", "lon"]]
display(weather_unique.sample(5))
display(weather_unique.shape)


,Station Name,lat,lon
235,LAGOON CITY,44.55,-79.22
292,MOOSE JAW A,50.33,-105.57
385,REGINA RCS,50.43,-104.67
524,WAINWRIGHT CFB AIRFIELD 21,52.83,-111.10
52,BUFFALO POUND LAKE,50.55,-105.38


(545, 3)

## Merge the production and weather station data

In [9]:
# filter out unique GeoUID and their prod_lat, prod_lon from province_prod, drop NaN
province_prod = pd.read_csv("./data/user_data/02_counterfactual_analysis/province_prod.csv")
province_prod_unique = province_prod.drop_duplicates(subset = ["GeoUID", "prod_lat", "prod_lon"]).dropna().reset_index(drop = True)[["provincename", "GeoUID", "prod_lat", "prod_lon"]]

display(province_prod_unique.head())
display(province_prod_unique.shape)

,provincename,GeoUID,prod_lat,prod_lon
0,Alberta,4801003,50.014229,-110.583589
1,Alberta,4801006,50.045816,-110.701488
2,Alberta,4801008,49.481525,-111.161699
3,Alberta,4801009,49.475704,-111.448178
4,Alberta,4801014,49.873328,-111.370904


(4475, 4)

In [10]:
# Match weather stations to closest GeoUID
# - 1. For each GeoUID, find the closest weather station.
#  * Each station can have multiple close GeoUIDs, but each GeoUID will have 
#    only one closest station.
# - 2. For each weather station in 1, take the most dominating provincename.
# - 3. Sum the GeoUIDs' productions over weather stations.


# STEP 1
# - Function to measure Eucleadian distance between two lat/lon pairs
def distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# For each GeoUID, find the closest station name and its lat, lon.
# - Add weather Station name, lat, lon, distance columns to a copy of province_prod_unique.

province_prod_unique2 = province_prod_unique.copy()
province_prod_unique2["weather_station_name"] = ""
province_prod_unique2["weather_station_lat"] = 0
province_prod_unique2["weather_station_lon"] = 0
province_prod_unique2["weather_station_distance"] = 0

for i in range(province_prod_unique2.shape[0]):
    
    # - get GeoUID, prod_lat, prod_lon
    geo_uid = province_prod_unique2.loc[i, "GeoUID"]
    prod_lat = province_prod_unique2.loc[i, "prod_lat"]
    prod_lon = province_prod_unique2.loc[i, "prod_lon"]
    
    # - get weather station name, lat, lon
    weather_station_name = weather_unique.loc[weather_unique.index[0], "Station Name"]
    weather_station_lat = weather_unique.loc[weather_unique.index[0], "lat"]
    weather_station_lon = weather_unique.loc[weather_unique.index[0], "lon"]
    
    # - compute distance
    weather_station_distance = distance(prod_lat, prod_lon, weather_station_lat, weather_station_lon)
    
    # - loop through weather_unique to find the closest station name, lat, lon
    for j in range(weather_unique.shape[0]):
        # j = 1
        
        # - get weather station name, lat, lon
        weather_station_name_temp = weather_unique.loc[weather_unique.index[j], "Station Name"]
        weather_station_lat_temp = weather_unique.loc[weather_unique.index[j], "lat"]
        weather_station_lon_temp = weather_unique.loc[weather_unique.index[j], "lon"]
        
        # - compute distance
        weather_station_distance_temp = distance(prod_lat, prod_lon, weather_station_lat_temp, weather_station_lon_temp)
        
        # - if distance is smaller, update weather_station_name, lat, lon, distance
        if weather_station_distance_temp < weather_station_distance:
            weather_station_name = weather_station_name_temp
            weather_station_lat = weather_station_lat_temp
            weather_station_lon = weather_station_lon_temp
            weather_station_distance = weather_station_distance_temp
    
    # - update province_prod_unique2
    province_prod_unique2.loc[i, "weather_station_name"] = weather_station_name
    province_prod_unique2.loc[i, "weather_station_lat"] = weather_station_lat
    province_prod_unique2.loc[i, "weather_station_lon"] = weather_station_lon
    province_prod_unique2.loc[i, "weather_station_distance"] = weather_station_distance


province_prod_unique2 = province_prod_unique2.dropna().reset_index(drop = True)
province_prod_unique2.head()
province_prod_unique2.shape

(4475, 8)

In [11]:
# STEP 2.
# - Extract weather_station_name and provincename
station_province = province_prod_unique2[["weather_station_name", "provincename"]].drop_duplicates().reset_index(drop = True)
station_province.shape

# - for each weather_station_name, compute the number of provincename
station_province["num_provincename"] = 0
for i in range(station_province.shape[0]):
    # i = 0
    weather_station_name = station_province.loc[i, "weather_station_name"]
    # weather_station_name
    num_provincename = station_province[station_province["weather_station_name"] == weather_station_name].shape[0]
    station_province.loc[i, "num_provincename"] = num_provincename

dup_provinces = station_province[station_province["num_provincename"] > 1]["weather_station_name"].tolist()

for station in dup_provinces:
    # station = dup_provinces[0]
    temp_dup = province_prod_unique2[province_prod_unique2["weather_station_name"] == station].groupby(["provincename"]).count().reset_index()
    dominant_prov = temp_dup[temp_dup["GeoUID"] == temp_dup["GeoUID"].max()]["provincename"].tolist()[0]

    # - drop all the rows with weather_station_name == station and provincename != dominant_prov
    province_prod_unique2 = province_prod_unique2[~((province_prod_unique2["weather_station_name"] == station) & (province_prod_unique2["provincename"] != dominant_prov))].reset_index(drop = True)

In [13]:
# - left join province_prod_unique2 to province_prod by GeoUID
province_prod.columns
province_prod_unique2.columns
province_prod2 = pd.merge(province_prod, province_prod_unique2[["GeoUID", "weather_station_name", "weather_station_distance"]], on = "GeoUID", how = "left")
province_prod2_cols1 = ["weather_station_name", "year", "provincename"]
province_prod2_cols2 = province_prod.columns[1:16].tolist()  # production data
province_prod2_cols = province_prod2_cols1 + province_prod2_cols2

In [26]:
# STEP 3.
# - group by weather_station_name and year, sum up all the columns in province_prod2_cols
province_prod3 = province_prod2[province_prod2_cols].dropna().groupby(["weather_station_name", "year", "provincename"]).sum().reset_index()
# print(province_prod3.shape)
# province_prod3.head()

# - merge province_prod4 to weather3 by weather_station_name and year
weather_prod_final = pd.merge(weather3, province_prod3, left_on = ["Station Name", "year"], right_on = ["weather_station_name", "year"], how = "left").dropna().reset_index().drop(columns = ["index", "Station Name"])
print(weather_prod_final.shape)
weather_prod_final.head()
weather_prod_final.tail()

(5301, 24)


,year,lat,lon,extreme_flag,tmax_flag,tmin_flag,total_flag,weather_station_name,provincename,X22.Utilities,X23.Construction,X31.33.Manufacturing,X48.49.Transportation.and.warehousing,X61.Educational.services,X62.Health.care.and.social.assistance,X72.Accommodation.and.food.services,X81.Other.services..except.public.administration.,X91.Public.administration,X11.Agriculture.forestry.fishing.hunting.21.Mining.quarrying.and.oil.and.gas.extraction,X41.Wholesale.trade.44.45.Retail.trade,X52.Finance.and.insurance.53.Real.estate.and.rental.and.leasing,X54.Professional..scientific.and.technical.services.55.56,X51.Information.culture.and.recreation.71,Population
5296,2002,51.44,-116.34,4,1,3,4,YOHO PARK,Alberta,0.0,0.734108,0.0,0.0,0.925170,0.462136,0.314625,0.382686,2.212805,3.089324,0.505408,0.0,0.729863,2.687369,1265.0
5297,2003,51.44,-116.34,1,0,1,1,YOHO PARK,Alberta,0.0,0.775217,0.0,0.0,0.865017,0.476920,0.315156,0.430354,2.112967,3.131059,0.558060,0.0,0.723913,2.516803,1265.0
5298,2004,51.44,-116.34,0,0,0,0,YOHO PARK,Alberta,0.0,0.842731,0.0,0.0,0.989087,0.513359,0.313377,0.424476,2.174512,3.312389,0.595165,0.0,0.737336,2.704839,1265.0
5299,2005,51.44,-116.34,1,1,1,2,YOHO PARK,Alberta,0.0,0.899558,0.0,0.0,0.990431,0.524977,0.338469,0.510992,2.248161,3.427475,0.639607,0.0,0.859000,2.642887,1265.0
5300,2006,51.44,-116.34,2,2,0,2,YOHO PARK,Alberta,0.0,1.091597,0.0,0.0,1.019785,0.513335,0.353394,0.493145,2.214093,3.469206,0.686344,0.0,0.879977,2.998478,1265.0


In [27]:
weather_prod_final.to_csv("./data/user_data/02_counterfactual_analysis/weather_prod_final.csv", index = False)

In [14]:
province_prod2.head()

,provincename,X22.Utilities,X23.Construction,X31.33.Manufacturing,X48.49.Transportation.and.warehousing,X61.Educational.services,X62.Health.care.and.social.assistance,X72.Accommodation.and.food.services,X81.Other.services..except.public.administration.,X91.Public.administration,X11.Agriculture.forestry.fishing.hunting.21.Mining.quarrying.and.oil.and.gas.extraction,X41.Wholesale.trade.44.45.Retail.trade,X52.Finance.and.insurance.53.Real.estate.and.rental.and.leasing,X54.Professional..scientific.and.technical.services.55.56,X51.Information.culture.and.recreation.71,Population,GeoUID,Dominant_NAICS,colourval,year,prod_lat,prod_lon,weather_station_name,weather_station_distance
0,Alberta,1.109974,18.154129,21.629247,12.574521,10.941785,13.597165,4.146254,6.591467,24.586121,394.898553,19.012793,25.804689,16.876098,8.646572,7662.0,4801003,X11.Agriculture.forestry.fishing.hunting.21.Mi...,#8DA0CB,1997,50.014229,-110.583589,ONEFOUR CDA,0.901415
1,Alberta,1.500429,17.595755,21.735976,14.462318,13.419916,14.276613,4.113031,5.917886,25.343374,368.275349,20.805581,27.013453,17.733267,8.839972,7662.0,4801003,X11.Agriculture.forestry.fishing.hunting.21.Mi...,#8DA0CB,1998,50.014229,-110.583589,ONEFOUR CDA,0.901415
2,Alberta,1.797365,19.861766,21.048838,15.004009,12.593374,15.291518,4.340496,6.851934,24.835660,381.344861,21.527044,27.302146,19.708950,8.378403,7662.0,4801003,X11.Agriculture.forestry.fishing.hunting.21.Mi...,#8DA0CB,1999,50.014229,-110.583589,ONEFOUR CDA,0.901415
3,Alberta,1.524674,22.414746,25.142960,15.663014,14.075712,15.862646,4.861173,6.756558,24.500474,390.238516,21.689333,29.912102,19.785181,9.211731,7662.0,4801003,X11.Agriculture.forestry.fishing.hunting.21.Mi...,#8DA0CB,2000,50.014229,-110.583589,ONEFOUR CDA,0.901415
4,Alberta,1.771950,22.129233,22.277156,15.681946,13.469488,17.226711,4.545922,7.772464,25.263720,372.023456,24.109734,31.765525,21.889195,10.188439,7662.0,4801003,X11.Agriculture.forestry.fishing.hunting.21.Mi...,#8DA0CB,2001,50.014229,-110.583589,ONEFOUR CDA,0.901415


In [27]:
# STEP 3-2.
weather_station_names = province_prod2.weather_station_name.unique().tolist()

closest_geouid_to_station = pd.DataFrame()
for station_name in weather_station_names:
    
    temp_weather = province_prod2.loc[province_prod2["weather_station_name"] == station_name, ["weather_station_name", "GeoUID", "weather_station_distance"]].drop_duplicates().sort_values(by = "weather_station_distance").reset_index(drop = True)
    print(temp_weather)
    # - take the first row and concat to closest_geouid_to_station
    closest_geouid_to_station = pd.concat([closest_geouid_to_station, temp_weather.loc[temp_weather.index[0], :].to_frame().transpose()], axis = 0)

closest_geouid_to_station
    

  weather_station_name   GeoUID  weather_station_distance
0          ONEFOUR CDA  4801003                  0.901415
  weather_station_name   GeoUID  weather_station_distance
0           BOW ISLAND  4801014                  0.163705
1           BOW ISLAND  4801009                  0.254303
2           BOW ISLAND  4801008                  0.380601
3           BOW ISLAND  4801018                  0.751763
4           BOW ISLAND  4801006                  0.812410
  weather_station_name   GeoUID  weather_station_distance
0           MILK RIVER  4802004                  0.042152
1           MILK RIVER  4802001                  0.153601
2           MILK RIVER  4802002                  0.154574
3           MILK RIVER  4802006                  0.219403
  weather_station_name   GeoUID  weather_station_distance
0       LETHBRIDGE CDA  4802012                  0.075084
1       LETHBRIDGE CDA  4802019                  0.154007
2       LETHBRIDGE CDA  4802013                  0.162045
3       LETHBR

IndexError: index 0 is out of bounds for axis 0 with size 0